In [1]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import  pandas as pd

[nltk_data] Downloading package gutenberg to /Users/utsav/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
#load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt','w') as file:
    file.write(data)

In [3]:
##laod the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()



#tokenization

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1 
total_words


4818

In [4]:
#input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
    

In [5]:
#pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')


In [6]:
#create predictors and label

X = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [7]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [18]:
#Define the model
#LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout , GRU

lstm_model = Sequential()
lstm_model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
lstm_model.add(LSTM(150, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100))
lstm_model.add(Dense(total_words, activation='softmax'))

#compile the model
lstm_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_3 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_3 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
#GRU RNN
#Define the model

gru_model=Sequential()
gru_model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
gru_model.add(GRU(150,return_sequences=True))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(100))
gru_model.add(Dense(total_words,activation="softmax"))

# #Compile the model
gru_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
gru_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 13, 100)           481800    
                                                                 
 gru_6 (GRU)                 (None, 13, 150)           113400    
                                                                 
 dropout_5 (Dropout)         (None, 13, 150)           0         
                                                                 
 gru_7 (GRU)                 (None, 100)               75600     
                                                                 
 dense_5 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1157418 (4.42 MB)
Trainable params: 1157418 (4.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
history_lstm = lstm_model.fit(
    X_train, y_train, 
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=[]
)



Epoch 1/50
644/644 [==============================] - 11s 18ms/step - loss: 6.3953 - accuracy: 0.0412 - val_loss: 7.0534 - val_accuracy: 0.0460
Epoch 2/50
644/644 [==============================] - 11s 17ms/step - loss: 6.1495 - accuracy: 0.0502 - val_loss: 7.1364 - val_accuracy: 0.0525
Epoch 3/50
644/644 [==============================] - 11s 17ms/step - loss: 6.0105 - accuracy: 0.0517 - val_loss: 7.1136 - val_accuracy: 0.0528
Epoch 4/50
644/644 [==============================] - 11s 17ms/step - loss: 5.8998 - accuracy: 0.0549 - val_loss: 7.2223 - val_accuracy: 0.0577
Epoch 5/50
644/644 [==============================] - 11s 18ms/step - loss: 5.7860 - accuracy: 0.0595 - val_loss: 7.2993 - val_accuracy: 0.0530
Epoch 6/50
644/644 [==============================] - 12s 18ms/step - loss: 5.6765 - accuracy: 0.0629 - val_loss: 7.3695 - val_accuracy: 0.0600
Epoch 7/50
644/644 [==============================] - 11s 18ms/step - loss: 5.5599 - accuracy: 0.0674 - val_loss: 7.4374 - val_accuracy:

In [24]:
history_gru = gru_model.fit(
    X_train, y_train, 
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=[]
)

Epoch 1/50
644/644 [==============================] - 13s 18ms/step - loss: 7.0800 - accuracy: 0.0315 - val_loss: 6.8241 - val_accuracy: 0.0251
Epoch 2/50
644/644 [==============================] - 11s 16ms/step - loss: 6.5329 - accuracy: 0.0400 - val_loss: 6.7888 - val_accuracy: 0.0472
Epoch 3/50
644/644 [==============================] - 10s 16ms/step - loss: 6.3200 - accuracy: 0.0489 - val_loss: 6.7935 - val_accuracy: 0.0507
Epoch 4/50
644/644 [==============================] - 11s 17ms/step - loss: 6.1093 - accuracy: 0.0629 - val_loss: 6.7467 - val_accuracy: 0.0678
Epoch 5/50
644/644 [==============================] - 10s 16ms/step - loss: 5.8835 - accuracy: 0.0761 - val_loss: 6.8170 - val_accuracy: 0.0707
Epoch 6/50
644/644 [==============================] - 11s 16ms/step - loss: 5.6598 - accuracy: 0.0848 - val_loss: 6.8971 - val_accuracy: 0.0711
Epoch 7/50
644/644 [==============================] - 11s 17ms/step - loss: 5.4482 - accuracy: 0.0937 - val_loss: 6.9513 - val_accuracy:

In [25]:
print("Evaluating LSTM...")
lstm_loss, lstm_acc = lstm_model.evaluate(X_test, y_test)

print("Evaluating GRU...")
gru_loss, gru_acc = gru_model.evaluate(X_test, y_test)

Evaluating LSTM...
161/161 [==============================] - 1s 6ms/step - loss: 11.4417 - accuracy: 0.0486
Evaluating GRU...
161/161 [==============================] - 1s 6ms/step - loss: 10.0035 - accuracy: 0.0511


In [26]:

gru_model.save("nextword_model.h5")
print("Saved GRU model as best_nextword_model.h5")

Saved GRU model as best_nextword_model.h5


/Users/utsav/Desktop/Project/next word prediction/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)